In [35]:
import natto

print("natto loaded")

natto loaded


In [36]:
from natto import MeCab

nm = MeCab()
text ="안녕하세요 저는 김민수입니다"

for node in nm.parse(text, as_nodes=True):
    print(node.surface, node.feature)

안녕 NNG,행위,T,안녕,*,*,*,*
하 XSV,*,F,하,*,*,*,*
세요 EP+EF,*,F,세요,Inflect,EP,EF,시/EP/*+어요/EF/*
저 NP,*,F,저,Inflect,NP,NP,제/NP/*
는 JX,*,T,는,*,*,*,*
김민수 NNP,인명,F,김민수,*,*,*,*
입니다 VCP+EC,*,F,입니다,Inflect,VCP,EC,이/VCP/*+ᄇ니다/EC/*
 BOS/EOS,*,*,*,*,*,*,*


In [37]:
#prepare doc object for spacy

In [38]:
#!pip install spacy

In [39]:
import json
import spacy
from spacy.tokens import Doc


In [42]:
nlp = spacy.blank("ko")

In [43]:
nlp.pipe_names

[]

In [44]:
with open("dlp_train_data.json", "r", encoding="utf-8") as f:
    TRAIN_DATA = json.load(f)

In [45]:
len(TRAIN_DATA)

200

In [46]:
TRAIN_DATA[0]

['김민수 매니저가 담당하는 베타는 내부 민감 프로젝트이다.',
 {'entities': [[14, 16, 'PROJECT_NAME'], [0, 3, 'PROJECT_MANAGER']]}]

In [49]:
type(TRAIN_DATA), type(TRAIN_DATA[0])

(list, list)

In [56]:
for i, item in enumerate(TRAIN_DATA[:5]):
    print(i, item[1]["entities"])


0 [[14, 16, 'PROJECT_NAME'], [0, 3, 'PROJECT_MANAGER']]
1 [[0, 6, 'PROJECT_NAME'], [13, 24, 'PROJECT_DATE'], [13, 24, 'PROJECT_DATE']]
2 [[0, 2, 'PROJECT_NAME'], [9, 21, 'PROJECT_DATE'], [24, 37, 'PROJECT_DATE']]
3 [[16, 23, 'PROJECT_NAME'], [30, 33, 'PROJECT_MANAGER'], [0, 5, 'ORG_INTERNAL']]
4 [[16, 18, 'PROJECT_NAME'], [25, 28, 'PROJECT_MANAGER'], [0, 5, 'ORG_INTERNAL']]


In [ ]:
from spacy.util import filter_spans

def create_spacy_docs(nlp, data):
    docs = []

    for item in data:
        text = item[0]
        entities = item[1]["entities"]

        doc = nlp.make_doc(text)
        spans = []

        for ent in entities:
            if len(ent) != 3:
                print("⚠️ Bad entity format:", ent)
                continue

            start, end, label = ent
            span = doc.char_span(start, end, label=label, alignment_mode="contract")

            if span is None:
                print("⚠️ Span error:", text[start:end], label)
            else:
                spans.append(span)

        # THIS LINE FIXES EVERYTHING avoid dupliate data
        doc.ents = filter_spans(spans)

        docs.append(doc)

    return docs


In [60]:
docs = create_spacy_docs(nlp, TRAIN_DATA)
len(docs)


200

In [61]:
doc = docs[1]
doc.text
[(ent.text, ent.label_) for ent in doc.ents]


[('데이터 통합', 'PROJECT_NAME'), ('2025년 3월 1일', 'PROJECT_DATE')]